In [ ]:

import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp
from genpen.utils import Paper
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from skimage import exposure

from skimage import filters
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.morphology import disk
from pyaxidraw import axidraw   # import module

from PIL import Image


import cv2

from genpen.flow.field import *
from genpen.flow.particle import *

import time

%load_ext autoreload
%autoreload 2

# spiral start buffer shaded

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'vp_test29.svg'
paper_size:str = '11x14 inches'
border:float=20  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)


In [ ]:
pt0 = Point(139.7, 70)

In [ ]:
c = pt0.buffer(70, resolution=20)

In [ ]:
buffer_gen = ss.uniform(loc=1, scale=1.1).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-1.3, high=-1.29)
d_translate_factor_gen = ss.uniform(loc=0.7, scale=0.1).rvs

In [ ]:
fills = []
all_polys = Polygon()
p = c.buffer(0.1, cap_style=2, join_style=3)
p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
angles_gen = gp.make_callable(np.radians(-90))
stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
stp.d_buffers += np.random.uniform(-0.05, 0.05, size=stp.d_buffers.shape)
P = gp.Poly(p)
P.fill_scale_trans(**stp.prms)
visible_area = p.difference(all_polys)
visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
visible_fill = gp.merge_LineStrings([g for g in reversed(visible_fill.geoms)])
fills.append(visible_fill)
all_polys = so.unary_union([all_polys, p])

In [ ]:
pt1 = Point(139.7, 211)

In [ ]:
c = pt1.buffer(70, cap_style=1, join_style=1, resolution=2)

In [ ]:
buffer_gen = ss.uniform(loc=1, scale=1.1).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-1.3, high=-1.29)
d_translate_factor_gen = ss.uniform(loc=0.7, scale=0.1).rvs

In [ ]:
visible_fill

In [ ]:

p = c
angles_gen = gp.make_callable(np.radians(90))
stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
stp.d_buffers += np.random.uniform(-0.05, 0.05, size=stp.d_buffers.shape)
P = gp.Poly(p)
P.fill_scale_trans(**stp.prms)
visible_area = p.difference(all_polys)
visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
visible_fill = gp.merge_LineStrings([g for g in reversed(visible_fill.geoms)])
fills.append(visible_fill)
all_polys = so.unary_union([all_polys, p])

In [ ]:
blss = gp.merge_LineStrings([f for f in fills if f.length > 0.1])

In [ ]:
blss = gp.make_like(blss, drawbox)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
# for ii, ls in enumerate(lss):
#     sk.stroke(ii + 1)
#     sk.geometry(ls)
sk.stroke(1)
sk.geometry(blss)
    
sk.display()

In [ ]:

ad = axidraw.AxiDraw()          # Initialize class
ad.interactive()
ad.options.units = 2
ad.connect()                    # Open serial port to AxiDraw 

In [ ]:
ad.options.speed_pendown = 60 

In [ ]:
ad.options.accel=50

In [ ]:
ad.update()

In [ ]:
blss.bounds

In [ ]:
def draw_geom(axidraw_handle, geom):
    pts = geom.coords
    y,x = pts[0]
    ad.moveto(x, y)
    for pt in pts[1:]:
        y,x = pt
        ad.lineto(x, y)
    ad.penup()

In [ ]:
wait_time = 2

In [ ]:
for geom in tqdm(blss):
    draw_geom(ad, geom)
    ad.moveto(0,0)
    time.sleep(wait_time)

In [ ]:
11 * 25.4

In [ ]:
pt = [0]

In [ ]:
pt

In [ ]:
                              # Absolute moves follow:
ad.moveto(356, 279)                  # Pen-up move to (1 inch, 1 inch)

In [ ]:
# ad.lineto(2,1)                  # Pen-down move, to (2 inch, 1 inch)
ad.moveto(0,0)  

In [ ]:
                # Pen-up move, back to origin.
ad.disconnect()                 # Close serial port to AxiDraw

# more complex

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'vp_test18.svg'
paper_size:str = '11x14 inches'
border:float=20  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)


In [ ]:
xgen = ss.uniform(loc=0.5, scale=0.05).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=1, 
    depth=0, 
    depth_limit=7,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
all_bps = gp.Shape(bps)

In [ ]:
# make pixel polys
prms = []
for bp in tqdm(bps):

#     a = np.random.uniform(0, 240)
    dist_from_center = bp.centroid.distance(bps.centroid)
    a = np.interp(dist_from_center, [0, 150], [0, 1020])
    prm = {
        'geometry':bp,
        'raw_pixel_width':pixel_width,
        'raw_pixel_height':pixel_height,
        'angle':a,
        'group': 'raw_hatch_pixel',
        'magnitude': np.random.uniform(0.3, 2),

    }
    prms.append(prm)
raw_hatch_pixels = geopandas.GeoDataFrame(prms)

In [ ]:
#  rescale polys to fit in drawbox
bbox = box(*raw_hatch_pixels.total_bounds)
_, transform = gp.make_like(bbox, drawbox, return_transform=True)
A = gp.AffineMatrix(**transform)
scaled_hatch_pixels = raw_hatch_pixels.copy()
scaled_hatch_pixels['geometry'] = scaled_hatch_pixels.affine_transform(A.A_flat)
scaled_hatch_pixels['scaled_pixel_height'] = scaled_hatch_pixels['geometry'].apply(gp.get_height)
scaled_hatch_pixels['scaled_pixel_width'] = scaled_hatch_pixels['geometry'].apply(gp.get_width)

In [ ]:
new_drawbox = so.unary_union(scaled_hatch_pixels.geometry)
db = gp.Poly(new_drawbox)

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels.geometry.centroid.y, [db.bottom, db.top], [0, 680]) + np.random.randn(len(scaled_hatch_pixels)) * 5

In [ ]:
scaled_hatch_pixels['angle'] = scaled_hatch_pixels['angle'] // 5 * 5

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels['angle'], xp=[0, 180], fp=[30, 150])

In [ ]:
qpg = NoisyQuantizedPiecewiseGrid(scaled_hatch_pixels, xstep=5, ystep=5, noise_scale=0.1, noise_mult=0.5, verbose=False)
qpg.make_grid()

In [ ]:
spiral_angle_max = np.pi * 200
spiral_angle_min = 0
spiral_angle_spacing = np.pi * 0.053
sp_angle_range = np.arange(spiral_angle_min, spiral_angle_max, spiral_angle_spacing)
spiral_distances = np.linspace(0, 100, len(sp_angle_range))

start_points = [Point(np.cos(a) * d, np.sin(a) * d) for a, d in zip(sp_angle_range, spiral_distances)]

In [ ]:
start_points = gp.make_like(MultiPoint(start_points), db.p)

In [ ]:
poly = new_drawbox
pts = []
lss = []
n_steps = 8
for pt in tqdm(start_points):
    
    vp = VectorParticle(pos=pt, grid=qpg, stepsize=1, momentum_factor=np.random.uniform(0,0))
    for ii in range(n_steps):
        vp.step()
    if len(vp.pts) > 1:
        ls = gp.merge_LineStrings([LineString(vp.pts)])
    lss.append(ls)

In [ ]:
buffer_gen = ss.uniform(loc=0.5, scale=0.5).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.35, high=-0.30)
d_translate_factor_gen = ss.uniform(loc=0.6, scale=0.8).rvs

In [ ]:
fills = []
all_polys = Polygon()
for ii, l in enumerate(tqdm(lss[:])):
    p = l.buffer(0.5, cap_style=2, join_style=2)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    angles_gen = gp.make_callable(sp_angle_range[ii])
#     angles_gen = gp.make_callable(-90)
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.05, 0.05, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
blss = gp.merge_LineStrings([f for f in fills if f.length > 0.1])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
sk.geometry(blss)
    
sk.display()

In [ ]:
25342 * 4 / 60 / 60

In [ ]:
len(blss)

In [ ]:
ii = 21

In [ ]:
ii % 20

In [ ]:
wait_time = 2

In [ ]:
blss

In [ ]:
def draw_geom_offset(axidraw_handle, geom, xoff=0, yoff=0):
    pts = geom.coords
    y,x = pts[0]
    ad.moveto(x+xoff, y+yoff)
    for pt in pts[1:]:
        y,x = pt
        ad.lineto(x+xoff, y+yoff)
    ad.penup()

In [ ]:

ad = axidraw.AxiDraw()          # Initialize class
ad.interactive()
ad.options.units = 2
ad.connect()                    # Open serial port to AxiDraw 

In [ ]:
ad.options.speed_pendown = 60 

In [ ]:
ad.options.accel=50

In [ ]:
ad.update()

In [ ]:
ad.moveto(0, 0)

In [ ]:
geom.length > 0

In [ ]:
yoff = 0
xoff = 0
counter = 0
for ii, geom in enumerate(tqdm(fills)):
    try:
        for line in geom:
            if line.length > 0:
                draw_geom_offset(ad, line, xoff=xoff, yoff=yoff)
                counter += 1
                if counter % 20 == 0:
                    ad.moveto(xoff, yoff)
                    time.sleep(wait_time)
    except:
        pass

In [ ]:
yoff = 0
xoff = 0
counter = 0
for ii, geom in enumerate(tqdm(fills)):
    for line in geom:
        if line.length > 0:
            draw_geom_offset(ad, line, xoff=xoff, yoff=yoff)
            counter += 1
            if counter % 20 == 0:
                ad.moveto(xoff, yoff)
                time.sleep(wait_time)

In [ ]:
len(fills)

# more complex

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'vp_test18.svg'
paper_size:str = '11x14 inches'
border:float=20  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)


In [ ]:
xgen = ss.uniform(loc=0.5, scale=0.25).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=1, 
    depth=0, 
    depth_limit=7,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
all_bps = gp.Shape(bps)

In [ ]:
# make pixel polys
prms = []
for bp in tqdm(bps):

#     a = np.random.uniform(0, 240)
    dist_from_center = bp.centroid.distance(bps.centroid)
    a = np.interp(dist_from_center, [0, 150], [0, 1020])
    prm = {
        'geometry':bp,
        'raw_pixel_width':pixel_width,
        'raw_pixel_height':pixel_height,
        'angle':a,
        'group': 'raw_hatch_pixel',
        'magnitude': np.random.uniform(0.3, 2),

    }
    prms.append(prm)
raw_hatch_pixels = geopandas.GeoDataFrame(prms)

In [ ]:
#  rescale polys to fit in drawbox
bbox = box(*raw_hatch_pixels.total_bounds)
_, transform = gp.make_like(bbox, drawbox, return_transform=True)
A = gp.AffineMatrix(**transform)
scaled_hatch_pixels = raw_hatch_pixels.copy()
scaled_hatch_pixels['geometry'] = scaled_hatch_pixels.affine_transform(A.A_flat)
scaled_hatch_pixels['scaled_pixel_height'] = scaled_hatch_pixels['geometry'].apply(gp.get_height)
scaled_hatch_pixels['scaled_pixel_width'] = scaled_hatch_pixels['geometry'].apply(gp.get_width)

In [ ]:
new_drawbox = so.unary_union(scaled_hatch_pixels.geometry)
db = gp.Poly(new_drawbox)

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels.geometry.centroid.y, [db.bottom, db.top], [0, 680]) + np.random.randn(len(scaled_hatch_pixels)) * 5

In [ ]:
scaled_hatch_pixels['angle'] = scaled_hatch_pixels['angle'] // 5 * 5

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels['angle'], xp=[0, 180], fp=[30, 150])

In [ ]:
qpg = NoisyQuantizedPiecewiseGrid(scaled_hatch_pixels, xstep=5, ystep=5, noise_scale=0.1, noise_mult=0.5, verbose=False)
qpg.make_grid()

In [ ]:
spiral_angle_max = np.pi * 200
spiral_angle_min = 0
spiral_angle_spacing = np.pi * 0.1
sp_angle_range = np.arange(spiral_angle_min, spiral_angle_max, spiral_angle_spacing)
spiral_distances = np.linspace(0, 100, len(sp_angle_range))

start_points = [Point(np.cos(a) * d, np.sin(a) * d) for a, d in zip(sp_angle_range, spiral_distances)]

In [ ]:
start_points = gp.make_like(MultiPoint(start_points), db.p)

In [ ]:
poly = new_drawbox
pts = []
lss = []
n_steps = 14
for pt in tqdm(start_points):
    
    vp = VectorParticle(pos=pt, grid=qpg, stepsize=1, momentum_factor=np.random.uniform(0,0))
    for ii in range(n_steps):
        vp.step()
    if len(vp.pts) > 1:
        ls = gp.merge_LineStrings([LineString(vp.pts)])
    lss.append(ls)

In [ ]:
buffer_gen = ss.uniform(loc=0.5, scale=0.5).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.4, high=-0.2)
d_translate_factor_gen = ss.uniform(loc=0.6, scale=0.8).rvs

In [ ]:
fills = []
all_polys = Polygon()
for ii, l in enumerate(tqdm(lss[:])):
    p = l.buffer(1.5, cap_style=2, join_style=2)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    angles_gen = gp.make_callable(sp_angle_range[ii])
#     angles_gen = gp.make_callable(-90)
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.05, 0.05, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
blss = gp.merge_LineStrings([f for f in fills if f.length > 0.1])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
sk.geometry(blss)
    
sk.display()

In [ ]:
save

In [ ]:
filename = 'vp_test30.svg'

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

In [ ]:
def draw_geom_offset(axidraw_handle, geom, xoff=0, yoff=0):
    pts = geom.coords
    y,x = pts[0]
    ad.moveto(x+xoff, y+yoff)
    for pt in pts[1:]:
        y,x = pt
        ad.lineto(x+xoff, y+yoff)
    ad.penup()

In [ ]:

ad = axidraw.AxiDraw()          # Initialize class
ad.interactive()
ad.options.units = 2
ad.connect()                    # Open serial port to AxiDraw 

In [ ]:
ad.options.speed_pendown = 60 

In [ ]:
ad.options.accel=50

In [ ]:
ad.update()

In [ ]:
ad.moveto(0, 0)

In [ ]:
geom.length > 0

In [ ]:
yoff = 0
xoff = 0
counter = 0
for ii, geom in enumerate(tqdm(fills)):
    try:
        for line in geom:
            if line.length > 0:
                draw_geom_offset(ad, line, xoff=xoff, yoff=yoff)
                counter += 1
                if counter % 20 == 0:
                    ad.moveto(xoff, yoff)
                    time.sleep(wait_time)
    except:
        pass

In [ ]:
yoff = 0
xoff = 0
counter = 0
for ii, geom in enumerate(tqdm(fills)):
    for line in geom:
        if line.length > 0:
            draw_geom_offset(ad, line, xoff=xoff, yoff=yoff)
            counter += 1
            if counter % 20 == 0:
                ad.moveto(xoff, yoff)
                time.sleep(wait_time)

In [ ]:
len(fills)

# bez shading

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'vp_test31.svg'
paper_size:str = '11x14 inches'
border:float=20  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)


In [ ]:
xgen = ss.uniform(loc=0.6, scale=0.1).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.7, 
    depth=0, 
    depth_limit=7,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:
layers = []
for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.1, 1.5)).rvs
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.25, 0.35),
            degrees=np.random.uniform(40,60),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

# sk.penWidth('0.2')
# for tolerance in [0.1, 0.3, 1.5]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm')
# sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
def draw_geom_offset(axidraw_handle, geom, xoff=0, yoff=0):
    pts = geom.coords
    y,x = pts[0]
    ad.moveto(x+xoff, y+yoff)
    for pt in pts[1:]:
        y,x = pt
        ad.lineto(x+xoff, y+yoff)
    ad.penup()

In [ ]:

ad = axidraw.AxiDraw()          # Initialize class
ad.interactive()
ad.options.units = 2
ad.connect()                    # Open serial port to AxiDraw 

In [ ]:
time.sleep(wait_time)

In [ ]:
ad.options.speed_pendown = 60 

In [ ]:
ad.options.accel=50

In [ ]:
ad.update()

In [ ]:
ad.moveto(0, 150)

In [ ]:
geom.length > 0

In [ ]:
ad.current_pos()

In [ ]:
ad.pen_lower()

In [ ]:
ad.pen_raise()

In [ ]:
yoff = 150
xoff = 0
counter = 0
wait_time=2.1
for ii, geom in enumerate(tqdm(fills[21:])):
    try:
        for line in geom:
            if line.length > 0:
                draw_geom_offset(ad, line, xoff=xoff, yoff=yoff)
                counter += 1
                if counter % 30 == 0:
                    ad.moveto(xoff, yoff)
                    time.sleep(wait_time)
    except KeyboardInterrupt:
        break
    except:
        pass

## testing

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'vp_test29.svg'
paper_size:str = '11x14 inches'
border:float=20  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)


In [ ]:
drawbox.bounds

In [ ]:
p = box(110, 30 ,180 ,110)

In [ ]:
p.bounds

In [ ]:
fills = []
xjitter_func = 0
yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.31, 0.35)).rvs
bhf = gp.BezierHatchFill(
    spacing=np.random.uniform(0.28, 0.3),
    degrees=90,
    poly_to_fill=p, 
    xjitter_func=xjitter_func, 
    yjitter_func=yjitter_func,
    fill_inscribe_buffer=1.4,
    n_nodes_per_line=10,
    n_eval_points=20,
)
fills.append(bhf.p)

In [ ]:
blss = gp.merge_LineStrings([f for f in fills if f.length > 0.1])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
sk.geometry(p.boundary)
for ii, ls in enumerate(blss):
    sk.stroke(ii + 2)
    sk.geometry(ls)
# sk.stroke(1)
# sk.geometry(blss)
sk.stroke(1000)
sk.geometry(drawbox.boundary)
    
sk.display()

In [ ]:
filename = 'vp_test31.svg'

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

In [ ]:
wait_time = 1.1

In [ ]:
len(blss)

In [ ]:
ad = axidraw.AxiDraw()   
ad.plot_setup(savepath)
ad.options.mode = "layers"

In [ ]:
for ii in tqdm(range(len(blss))):
    ad.options.layer = ii
    ad.options.speed_pendown = 90
    ad.plot_run()
    time.sleep(wait_time)
    

In [ ]:

ad = axidraw.AxiDraw()          # Initialize class
ad.interactive()
ad.options.units = 2
ad.connect()                    # Open serial port to AxiDraw 

In [ ]:
ad.options

In [ ]:
ad.options.speed_pendown = 90 

In [ ]:
ad.options.accel=90

In [ ]:
ad.update()

In [ ]:
def draw_geom(axidraw_handle, geom):
    pts = geom.coords
    y,x = pts[0]
    ad.moveto(x, y)
    for pt in pts[1:]:
        y,x = pt
        ad.lineto(x, y)
    ad.penup()

In [ ]:
ii = 2
geom = blss[ii]

In [ ]:
draw_geom(ad, geom)

In [ ]:
for geom in tqdm(blss):
    draw_geom(ad, geom)
#     ad.moveto(0,0)
#     time.sleep(wait_time)

In [ ]:
11 * 25.4

In [ ]:
pt = [0]

In [ ]:
ad.penup()

In [ ]:
                              # Absolute moves follow:
ad.moveto(30, 110)                  # Pen-up move to (1 inch, 1 inch)

In [ ]:
# ad.lineto(2,1)                  # Pen-down move, to (2 inch, 1 inch)
ad.moveto(0,0)  

In [ ]:
                # Pen-up move, back to origin.
ad.disconnect()                 # Close serial port to AxiDraw